# Exercícios Pandas Parte 2 # 

In [2]:
import pandas as pd
import numpy as np


## Exercício 1 ##
Olhando apenas para o grupo de divorciados, qual o percentual deles que é empreendedor (entrepreneur)? E estudantes?.</p>


In [3]:
bank_df= pd.read_csv('bank-full.csv', sep=';')
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Resolução

<p> Primeiro, vou verificar quantos tipos únicos temos de jobs e estados cívis.</p>

In [4]:
#verificando tipos únicos
bank_df.job.unique()
bank_df.marital.unique()

array(['married', 'single', 'divorced'], dtype=object)

<p> Para job temos as seguintes categorias: </p>
<ul> management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student </ul>
<p> E para marital temos as seguintes categorias:
<ul>['married', 'single', 'divorced'] </ul>

In [5]:
#filtrando divorciados e estudantes que sao divorciados
quantidade= bank_df.query('job in[ "entrepreneur", "student"] and marital == "divorced"')

#contando a quantidade de pessoas que são divorciadas e ou estudantes ou empresarios
qtd= quantidade.shape[0]

#quantidade de divorciados
divorciados= bank_df.query('marital == "divorced"')
qtde_divorciados= divorciados.shape[0]

#percentual de divorciados que são estudantes ou empresários
percentual= (qtd/qtde_divorciados) *100

print(f'O número de pessoas que são divorciadas e são ou estudante ou empresários é de {qtd}')
print(f' O total dos divorciados é de {qtde_divorciados}')
print(f' E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de {percentual:.2f} %')

#obtendo o percentual

O número de pessoas que são divorciadas e são ou estudante ou empresários é de 185
 O total dos divorciados é de 5207
 E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de 3.55 %


Talvez tenha melhor forma de fazer isso, mas seguimos em frente. O importante é continuar.
</details>

## Exercício 2 ##
 O seu chefe decidiu criar duas categorias, Premium e Gold, para focar nos clientes mais promissores. Para a categoria Premium, entrarão os que possuem saldo (balance) nos top 10%. Ou seja, o grupo acima do percentil 90. Já o grupo Gold, ficarão os que possuem saldo nos top 20%, mas abaixo do grupo Premium. Ou seja, do p80 até o último do p90. Verifique quantos clientes possuem no grupo Premium e no Gold.

Primeiramente, iremos calcular o percentil de 90% e 80% do balance:

grupo premium: 10 % com mais balance, percentil de 90% 
grupo gold: percentil de 80% até 90%

categoria defina pelo balance

In [6]:
# definindo percentil de 80%
percent_80= bank_df['balance'].quantile(0.80)

#definindo percentil de 90%
percent_90= bank_df['balance'].quantile(0.90)

percent_80
percent_90

#definindo intervalos para o cut

bins= [bank_df['balance'].min(), percent_80, percent_90, bank_df['balance'].max()]
labels=['gold', 'premium', 'top 10%']

#categorizando os dados

bank_df['categoria']=pd.cut(bank_df['balance'], bins=bins, labels=labels, include_lowest= True)

#contando quantos clientes estão em cada grupo
contagem= bank_df['categoria'].value_counts()

print(bank_df)
print(f"O número de clientes na categoria Gold : {contagem.get('gold',0)}")
print(f"O número de clientes na categoria Premium: {contagem.get('premium',0)}")

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd

Versão 2
Primeiro, vamos calcular também o percentil de 90 e 80


In [7]:
#Calculando o percentil de 90 e 80

percentiu_90= np.percentile(bank_df['balance'],90)
percentiu_80= np.percentile(bank_df['balance'],80)

#Criando coluna para categoria Premium

bank_df['CATEGORIA']= np.where(bank_df['balance'] >= percentiu_90, 'Premium',
                               np.where(bank_df['balance']>= percentiu_80, 'Gold', 'Outros'))

#np Where cria novas tabelas com base nas condições

#Contando os Clientes em cada categoria

contagem_categoria= bank_df['CATEGORIA'].value_counts()
print(contagem_categoria)

Outros     36162
Gold        4525
Premium     4524
Name: CATEGORIA, dtype: int64


A primeira solução é mais flexível e a segunda é mais fácil e intuitiva.

## Exercício 3 ##

3) Seu chefe também quer saber se existe diferença nos percentuais vistos no exercício (1) para o grupo Premium. Verifique como a tabela cruzada da população se diferencia do grupo Premium. Existe diferença? Qual sua hipótese para que exista essa diferença?

In [8]:
#Selecionando apenas os premium na coluna de categoria
premium_df = bank_df[bank_df['CATEGORIA'] == 'Premium']
premium_df

#Tabela cruzada para população total
crosstab_total= pd.crosstab(bank_df['job'], bank_df['marital'], normalize= 'index')

#tabela cruzada de job apenas para categoria premium
crosstab_premium= pd.crosstab(premium_df['job'], premium_df['marital'], normalize= 'index')

#comparando percentuais
print(crosstab_total)
print(crosstab_premium)

marital        divorced   married    single
job                                        
admin.         0.145040  0.520789  0.334171
blue-collar    0.077065  0.715988  0.206946
entrepreneur   0.120377  0.719570  0.160054
housemaid      0.148387  0.735484  0.116129
management     0.117467  0.570945  0.311588
retired        0.187721  0.764576  0.047703
self-employed  0.088664  0.628879  0.282457
services       0.132162  0.579442  0.288397
student        0.006397  0.057569  0.936034
technician     0.121759  0.533368  0.344873
unemployed     0.131236  0.561013  0.307751
unknown        0.059028  0.704861  0.236111
marital        divorced   married    single
job                                        
admin.         0.100478  0.612440  0.287081
blue-collar    0.056604  0.723720  0.219677
entrepreneur   0.083916  0.825175  0.090909
housemaid      0.171642  0.701493  0.126866
management     0.104361  0.598131  0.297508
retired        0.160237  0.818991  0.020772
self-employed  0.134409  0.61828

A diferença encontrada foi de que entre o grupo de premiuns, o percentual de empresários e que são casados, são bem superiores ao restante da população. Há também mais empresários casados do que solteiros. Acredito que isso seja esperado, por talvez serem homens com mais idade, já vivendo o matrimônio.

### exercício 4 ###

4) Para a apresentação final, você decide criar uma tabela com os 10 clientes que possuem maior saldo (balance). Como a tabela é grande demais, você resolve apresentar apenas as informações relevantes: age, job, marital, education, default, loan, duration. Além disso, você decide destacar clientes que foram mais e menos atenciosos, pois são os que parecem ter maior e menor potencial, respectivamente. Para isso, destaque na tabela, de amarelo, o cliente de maior duration. Destaque, de vermelho, o que tem menor duration.


In [9]:
#selecionando as tabelas que quero do dataframe
select_tab= ['balance','age', 'job', 'marital', 'education', 'default', 'loan', 'duration']

#criando um dataframe filtrado
filtered_bankdf= bank_df[select_tab]

#selecionando os top 10 com maiorees balances
top_10balance= filtered_bankdf.nlargest(10, 'balance')
print(top_10balance)

# Função para aplicar cores aos valores máximos e mínimos na coluna 'duration'
def highlight_max_min(data):
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    max_val = data['duration'].max()
    min_val = data['duration'].min()
    
    # Aplicando o estilo
    styles['duration'] = ['background-color: yellow' if v == max_val else 'background-color: red' if v == min_val else '' for v in data['duration']]
    
    return styles

# Aplicando a função ao DataFrame
styled_df = top_10balance.style.apply(highlight_max_min, axis=None)

# Exibindo o DataFrame estilizado
styled_df

       balance  age           job   marital  education default loan  duration
39989   102127   51    management    single   tertiary      no   no        90
26227    98417   59    management   married   tertiary      no   no       145
42558    81204   84       retired   married  secondary      no   no       679
43393    81204   84       retired   married  secondary      no   no       390
41693    71188   60       retired   married    primary      no   no       205
19785    66721   56    management  divorced   tertiary      no   no       442
21192    66653   52   blue-collar   married    primary      no   no       109
19420    64343   59        admin.   married    unknown      no   no        45
41374    59649   32  entrepreneur    single   tertiary      no   no        69
12926    58932   56   blue-collar   married  secondary      no   no       339


,balance,age,job,marital,education,default,loan,duration
39989,102127,51,management,single,tertiary,no,no,90
26227,98417,59,management,married,tertiary,no,no,145
42558,81204,84,retired,married,secondary,no,no,679
43393,81204,84,retired,married,secondary,no,no,390
41693,71188,60,retired,married,primary,no,no,205
19785,66721,56,management,divorced,tertiary,no,no,442
21192,66653,52,blue-collar,married,primary,no,no,109
19420,64343,59,admin.,married,unknown,no,no,45
41374,59649,32,entrepreneur,single,tertiary,no,no,69
12926,58932,56,blue-collar,married,secondary,no,no,339


### Exercício 5 ###


5) Na sua opinião e de parte da diretoria, há 3 fatores que determinam se a pessoa vai aceitar a proposta do seu banco (y): o saldo (balance), se ela já deixou de pagar um empréstimo (default) e se ela tem um empréstimo pessoal (loan). Para avaliar essas 3 características junto com o aceite (y), você precisará criar uma tabela dinâmica, cujo valor observado seja a média do saldo. Avalie se existe diferença no saldo de quem aceita a proposta e tem empréstimo vs quem não aceita e não tem empréstimo, e o mesmo para o default. Em outras palavras, crie uma tabela dinâmica cruzando loan e default, mas também segmentando por aceite ou não da proposta. O saldo de quem aceita e tem empréstimo é maior ou menor do que o de quem não aceita? Por qual motivo você acha que isso acontece? Crie a sua hipótese para justificar o resultado achado!

Dica: Crie uma tabela dinâmica cujo índice seja a coluna y e default (ou loan), o campo columns seja loan (ou default) e os valores sejam a média do saldo (balance).

Y: vai aceitar a proposta do banco
balance: saldo
default: se ela deixoude pagar um emprestimo
loan: se ela tem um emprestimo pessoal.

média do balance. Tem diferença no saldo de quem aceita a proposta vs quem não aceita e não tem emprestimo?


In [10]:
# selecionando as variáveis e criando um filtro pro dataset
variaveis = ['balance', 'y', 'default', 'loan']
filtro_vav= bank_df[variaveis]

#tabela dinâmica
pivot_table = bank_df.pivot_table(
    index=['y', 'default'], #linhas da tabela: aceitou a proposta e deixou de pagar emprestimo
    columns='loan', #colunas da tabela: possui algum empréstimo pessoal?
    values='balance', #valor do saldo
    aggfunc= 'mean' #função agregada: média
)
print(pivot_table)

loan                  no         yes
y   default                         
no  no       1435.673581  807.983318
    yes      -124.737945 -172.958042
yes no       1912.359060  912.245203
    yes       -82.108108  -10.666667


### Análise dos Resultados ###

**Interpretação da Tabela:**

A coluna "yes" ou "no" indica se as pessoas possuem empréstimo ou não. Os valores representam a média do saldo.

As linhas refletem a combinação de duas variáveis: y (aceitou a proposta) e default (já deixou de pagar um empréstimo).

* **Grupo 1:** Pessoas que não aceitaram a proposta e não têm dívidas. O saldo é maior para quem não possui empréstimo.
* **Grupo 2:** Pessoas que não aceitaram a proposta e têm dívidas. O saldo é negativo para ambos os grupos (com e sem empréstimo).
* **Grupo 3:** Pessoas que aceitaram a proposta e não têm dívidas. O maior saldo é observado neste grupo, especialmente para quem não possui empréstimo.
* **Grupo 4:** Pessoas que aceitaram a proposta e têm dívidas. O saldo negativo é maior para quem não possui empréstimo.

**Respondendo à pergunta chave:**

O saldo de quem aceita e tem empréstimo (912.245203) é menor do que o de quem não aceita e não tem empréstimo (1435.673581).

**Hipótese:**

Pessoas com saldos altos e sem dívidas podem ser mais avessas a riscos e preferem manter suas economias em vez de contrair novos empréstimos. Por outro lado, pessoas com saldos menores e histórico de dívidas podem buscar novas linhas de crédito para consolidar dívidas ou financiar projetos.

**Observações:**

* **Saldo de quem aceita e não tem empréstimo:** O alto valor (1912.359060) indica que este grupo possui um bom perfil financeiro e pode ser um alvo interessante para o banco.
* **Saldos negativos:** Sugerem que essas pessoas podem estar com dificuldades financeiras.

**Próximos passos:**

* **Análise gráfica:** Visualizar os dados em gráficos pode facilitar a compreensão.
* **Segmentação de clientes:** Dividir os clientes em grupos com características semelhantes para uma análise mais detalhada.
* **Modelagem preditiva:** Construir modelos para prever a probabilidade de um cliente aceitar a proposta.

### Exercício 6 ###

**6)** Usando a tabela criada no exercício (5), qual o quadrante que possui menor saldo dentre os 8 da tabela dinâmica. Por qual motivo você acredita que o saldo dele é o menor de todos?

O menor saldo, é o saldo negativo de -172.958042 e pertence ao grupo que não aceitou a proposta do banco, mas possuem empréstimos. Uma justificativa plausível, é que se trata de um grupo de pessoas inadiplentes com problemas financeiros e que buscou empréstimos para regularizar sua situação financeira ou tentar realizar mais investimentos, mas que atualmente enfrenta problemas para arcar com compromissos, uma vez que também fazem parte do grupo que também já realizaram algum empréstimo anteriormente.

### Exercício 7 ##

**7)** Você trabalha numa empresa de bicicletas e precisa fazer uma clusterização separando marcas caras de baratas. Para isso, você precisará saber a média de preços das bicicletas de cada marca.

Dica: No dataset brands, você encontra as marcas e no products você possui o preço de cada bicicleta. Traga a informação de preço de products para brands usando a coluna brand_id, e, em seguida, faça o agrupamento de preço médio por cada brand.

**São 2 datasets diferentes, por isso precisaremos utilzar join**

In [14]:
# 1 data set brands
#2 dataset products
# on or id = brand ID

brands_df=pd.read_csv('brands.csv')
brands_df.info

products_df= pd.read_csv('products.csv')
products_df

#juntando os dois datasets
analise_brand= brands_df.merge(products_df, on='brand_id', how='left')
analise_brand

#agrupando preço por brand
analise_groupbrand= analise_brand.groupby('brand_name')['list_price'].sum().reset_index()
analise_groupbrand

#aplicando função de max e min
# Função para aplicar cores aos valores máximos e mínimos na coluna 'duration'
def highlight_max_min(data):
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    max_val = data['list_price'].max()
    min_val = data['list_price'].min()
    
    # Aplicando o estilo
    styles['list_price'] = ['background-color: yellow' if v == max_val else 'background-color: red' if v == min_val else '' for v in data['list_price']]
    
    return styles

# Aplicando a função ao DataFrame
styled_df = analise_groupbrand.style.apply(highlight_max_min, axis=None)

# Exibindo o DataFrame estilizado
styled_df


,brand_name,list_price
0,Electra,89798.730000
1,Haro,6219.900000
2,Heller,6518.990000
3,Pure Cycles,1327.000000
4,Ritchey,749.990000
5,Strider,629.970000
6,Sun Bicycles,12062.770000
7,Surly,33293.840000
8,Trek,337508.650000


## Exercício 8 ##

**8)** Agora que você já possui o preço médio de cada marca, você deve separar os valores em 3 grupos, sendo que os nomes de cada agrupamento será a partir dessa lista: ["Barato", "Médio", "Caro"].

Dica: Será que cut ou qcut podem resolver este problema para você?

In [17]:
# 1 data set brands
#2 dataset products
# on or id = brand ID

brands_df=pd.read_csv('brands.csv')
brands_df.info

products_df= pd.read_csv('products.csv')
products_df

#juntando os dois datasets
analise_brand= brands_df.merge(products_df, on='brand_id', how='left')
analise_brand

#agrupando preço por brand
analise_groupbrand= analise_brand.groupby('brand_name')['list_price'].sum().reset_index()
analise_groupbrand

#divindo em grupos com qcut

analise_groupbrand['categoria_vendas']= pd.qcut(analise_groupbrand['list_price'], q=3, labels=['Baixo', 'Médio', 'Alto'])
print(analise_groupbrand)



     brand_name  list_price categoria_vendas
0       Electra    89798.73             Alto
1          Haro     6219.90            Médio
2        Heller     6518.99            Médio
3   Pure Cycles     1327.00            Baixo
4       Ritchey      749.99            Baixo
5       Strider      629.97            Baixo
6  Sun Bicycles    12062.77            Médio
7         Surly    33293.84             Alto
8          Trek   337508.65             Alto


## Exercício 9 ##
**9)** Outra demanda chegou, dessa vez precisamos entender se o preço tem alguma relação com as vendas. Bicicletas mais caras são mais vendidas que as mais baratas? Qual a diferença nas vendas entre cada um dos grupos que levantamos na última pergunta?

**Dica:** Agora, você precisará partir do DataFrame brands, trazer informações de produtos de cada marca e depois ir na order_items para trazer a quantidade de cada produto vendido. O primeiro join será com a coluna brand_id que é comum ao dataFrame brands e ao products. Na sequência, você consegue ver qual coluna existe em products e em order_items para fazermos mais um join? E o último agrupamento, como será feito? A pergunta é quanto cada marca vende, ou seja, o último agrupamento só pode ser com a coluna br...

**Observação**: Pensar em uma solução mais tarde, pois a principio não parece ter colunas em comum em todos os datasets.

In [42]:
#importando data set ordes
orders_itensdf= pd.read_csv('order_items.csv')
#juntando os dois primeiros datasets
analise_sale= brands_df.merge(products_df, on='brand_id', how='left')

#juntando o último dataframe
analise_sale_final= analise_sale.merge(orders_itensdf, on= 'product_id', how= 'left')
analise_sale_final

#group by vendas por marcas
resultado= analise_sale_final.groupby('brand_name')['list_price_x'].mean().reset_index
print(resultado)


<bound method Series.reset_index of brand_name
Electra          520.365237
Haro             629.262727
Heller          1400.042165
Pure Cycles      441.945736
Ritchey          749.990000
Strider          194.990000
Sun Bicycles     529.408557
Surly           1168.135611
Trek            2762.359478
Name: list_price_x, dtype: float64>


## Exercício 10 ##

**10)** Uma das coisas mais importantes para sua marca é expandir pelo maior número de estados possíveis. Para isso, precisamos entender quais estados consomem pouco e quais estados consomem muitos produtos. Em outras palavras, precisamos entender de onde vem a receita e onde precisamos gastar mais com marketing, ou entender se seria melhor simplesmente desligar a operação no local. Crie uma tabela sumarizando os gastos por estado, verifique qual a diferença do estado que mais consome para o estado que menos consome.

Sem dica para a última tarefa!

In [35]:
#primeiro carregando todas os dataframes e analisando os nomes de coluna

categories_df= pd.read_csv('categories.csv')
orders_itensdf= pd.read_csv('order_items.csv')
orders_df=pd.read_csv('orders.csv')
staff_df= pd.read_csv('staffs.csv')
stocks_df= pd.read_csv('stocks.csv')
stores_df=pd.read_csv('stores.csv')


display(brands_df.columns)
display(products_df.columns)
display(orders_itensdf.columns)
display(orders_df.columns)
display(staff_df.columns)
display(stocks_df.columns)
display(stores_df.columns)

#iremos utilizar o products + stocks+ stores df

Index(['brand_id', 'brand_name'], dtype='object')

Index(['product_id', 'product_name', 'brand_id', 'category_id', 'model_year',
       'list_price'],
      dtype='object')

Index(['order_id', 'item_id', 'product_id', 'quantity', 'list_price',
       'discount'],
      dtype='object')

Index(['order_id', 'customer_id', 'order_status', 'order_date',
       'required_date', 'shipped_date', 'store_id', 'staff_id'],
      dtype='object')

Index(['staff_id', 'first_name', 'last_name', 'email', 'phone', 'active',
       'store_id', 'manager_id'],
      dtype='object')

Index(['store_id', 'product_id', 'quantity'], dtype='object')

Index(['store_id', 'store_name', 'phone', 'email', 'street', 'city', 'state',
       'zip_code'],
      dtype='object')

In [53]:


# Merge para obter preço, quantidade e id da loja
analise_state = products_df.merge(stocks_df, on='product_id', how='left')

# Merge para sabermos de qual estado é a loja
analise_state_final = analise_state.merge(stores_df, on='store_id', how='left')

# Nova coluna total de vendas
analise_state_final['total_de_vendas'] = analise_state_final['list_price'] * analise_state_final['quantity']

# Agrupando os estados e somando as vendas totais
sales_by_state = analise_state_final.groupby('state')['total_de_vendas'].sum().reset_index()

# Ordenar resultados por ordem decrescente
sales_by_state = sales_by_state.sort_values(by='total_de_vendas', ascending=False)

print(sales_by_state)


  state  total_de_vendas
2    TX       6859214.21
0    CA       6487242.16
1    NY       6473126.28
